<a href="https://colab.research.google.com/github/rishi-a/Twitter-AQ/blob/master/Hypothesis%201/BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import balanced_accuracy_score
from matplotlib import pyplot as plt
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
import numpy
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.constraints import maxnorm
import pickle 
#pd.set_option('display.max_colwidth', 2000) # To visualize full output
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
import pickle 
#dum = pd.read_csv('https://raw.githubusercontent.com/patel-zeel/kdd_data_open/master/smogtowertweets%2B150.csv')
#url = 'https://raw.githubusercontent.com/patel-zeel/kdd_data_open/master/SmogTweet500%20-%20210TZ.csv'
df_main = pd.read_csv('OddEven.csv')
#frames = [df_main,dum]
#df_main = pd.concat(frames)
#df_main = df_main.loc[df_main['TLabel'] == df_main['ZLabel']]
df_main.shape
#df_main = df_main.sample(len(df_main), random_state=42)

In [ ]:
def preprocess(tweet):
  #tweet = tweet.replace('@ ','@').replace('# ','#')
  return ' '.join(w for w in tweet.split() if w[0] not in ['@'] and 
                  not w.startswith('pic.twitter.com') and
                  not w.startswith('http://') and
                  not w.startswith('https://')).lower()

In [ ]:
df = df_main
df['Tweet text processed'] = df['tweet'].apply(preprocess)

In [ ]:
print('before',len(df))
df = df.drop_duplicates(subset='Tweet text processed')
print('after',len(df))

In [ ]:
ax = df['ZLabel'].value_counts().plot(kind='bar')
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))

In [ ]:
df = df.reset_index(drop = True)
df.head()

In [ ]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Embedding, LSTM, GRU 
from tensorflow.keras.layers import Dense, Embedding, LSTM, GRU, Bidirectional
from sklearn.metrics import classification_report, balanced_accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

toc = Tokenizer()
toc.fit_on_texts(df['Tweet text processed'])

max_len = max(len(s.split()) for s in df['Tweet text processed'])

vocab_size = len(toc.word_index) + 1

In [ ]:
from itertools import product
from sklearn.metrics import balanced_accuracy_score

def create_model(embedding_dim, neurons):
	# create model
  model = Sequential()
  model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
  model.add(Bidirectional(LSTM(32, return_sequences=True)))
  model.add((Bidirectional(LSTM(neurons))))
  #model.add(Dense(64, activation='relu'))
  #model.add(Dense(3, activation='relu'))
  model.add(Dense(3, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
def GridSearchCV(X, y, train_val_index, y_train_val, param_grid):
  hyperpara_dict = {}
  kfInner = StratifiedKFold(5, shuffle=True, random_state=0)
  train_ind, val_ind = next(kfInner.split(train_val_index, y_train_val.idxmax(axis=1).values))
  train_df_ind = train_val_index[train_ind]
  val_df_ind = train_val_index[val_ind]
  X_train, y_train = X.iloc[train_df_ind], y.iloc[train_df_ind]
  X_val, y_val = X.iloc[val_df_ind], y.iloc[val_df_ind]

  train_tocs = toc.texts_to_sequences(X_train)
  val_tocs = toc.texts_to_sequences(X_val)
  X_train_pad = pad_sequences(train_tocs, maxlen=max_len, padding='post')
  X_val_pad = pad_sequences(val_tocs, maxlen=max_len, padding='post')
  
  for embedding_dim, neurons, batch_size, epochs in product(*param_grid.values()):
    print('searching', (embedding_dim, neurons, batch_size, epochs))
    model = create_model(embedding_dim, neurons)
    model.fit(X_train_pad, y_train, batch_size=batch_size, epochs=epochs, verbose=0)
    res = model.predict(X_val_pad)
    score = pd.DataFrame(res).max(axis =1)
    y_pred = pd.DataFrame(res, columns=[-1,0,1]).idxmax(axis=1)
    accuracy = f1_score(y_val.idxmax(axis=1), y_pred,average='macro')
    hyperpara_dict.update({(embedding_dim, neurons, batch_size, epochs) : accuracy})
  return max(hyperpara_dict, key=lambda x: hyperpara_dict[x]) # will return set of hyp having max accuracy


In [ ]:
# Main code
from sklearn.model_selection import train_test_split
X = df['Tweet text processed']
y = pd.get_dummies(df['ZLabel'])
param_grid = {'embedding_dim':[32,64,128], 
               'neurons':[8,16,32,64], 
               'batch_size':[8,16,32], 
               'epochs' : [20, 30,50, 100]}
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold()
test_inds = []
test_preds = []
test_score = []
test = []
for train_val_index, test_index in kf.split(X, y.idxmax(axis=1)):
  X_train_val, X_test = X.iloc[train_val_index], X.iloc[test_index]
  y_train_val, y_test = y.iloc[train_val_index], y.iloc[test_index]
  train_val_tocs = toc.texts_to_sequences(X_train_val)
  test_tocs = toc.texts_to_sequences(X_test)
  X_train_val_pad = pad_sequences(train_val_tocs, maxlen=max_len, padding='post')
  X_test_pad = pad_sequences(test_tocs, maxlen=max_len, padding='post')

  # Start Hyperpara tuning
  embedding_dim, neurons, batch_size, epochs = GridSearchCV(X, y, train_val_index, y_train_val, param_grid)
  print('best hyper', embedding_dim, neurons, batch_size, epochs)
  model = create_model(embedding_dim, neurons)
  model.fit(X_train_val_pad, y_train_val, batch_size=batch_size, epochs=epochs, verbose=0,callbacks=[LogThirdLayerOutput()])
  res = model.predict(X_test_pad)
  score = pd.DataFrame(res).max(axis =1)
  y_pred = pd.DataFrame(res, columns=[-1,0,1]).idxmax(axis=1)
  accuracy = f1_score(y_test.idxmax(axis=1), y_pred,average='macro')
  print(accuracy)
  test_inds.extend(test_index)
  test_preds.extend(y_pred)
  test_score.extend(score)
  test.extend(y_test.idxmax(axis=1))

In [ ]:
print(classification_report(test, test_preds))
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, balanced_accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
print(test_preds)
sns.heatmap((confusion_matrix(test,test_preds)),annot=True,annot_kws={"size": 40},cbar=False,fmt='g',
            xticklabels=[-1,0,1], yticklabels=[-1,0,1])

plt.xlabel('predicted')
plt.ylabel('ground truth')
plt.show()

In [ ]:
print(test_inds)
print(test)
print(test_preds)
print(test_score)

In [ ]:
df = df.reindex(test_inds)
df.shape

In [ ]:
df['prediction'] = test_preds
df['score'] = test_score

In [ ]:
result = df[['Tweet text processed','ZLabel','prediction','score']]
result.to_csv('3bilstmwithoutoddfinal.csv')

In [ ]:
result[result['TLabel'] != result['prediction']]